In [1]:
import altair as alt
import itertools
import pandas as pd

from fairness_utils import *
from latextable import draw_latex
from texttable import Texttable
from scorer import *

FILENAME = 'data/compas_predictions.csv'
PREDICTOR_TYPES = ['unconstrained','calibrated','thresholdless_EO']
RACES = ['all','African-American','Caucasian']

In [2]:
df = pd.read_csv(FILENAME)

In [3]:
# Fairness and accuracy criteria
table = Texttable()
table.set_deco(Texttable.HEADER)
table.set_cols_dtype([
    't',
    't',
    'f',
    'f',
    'f',
    'f',
    'f',
])
table.add_row(['Predictor','Race','Brier Score','Log Loss','Calibration Gap','Separation Gap (Negative Class)','Separation Gap (Positive Class)'])
brier_accuracy_scorer = WeightedBrierScorer()
log_loss_scorer = WeightedLogScorer()
for predictor in PREDICTOR_TYPES:
    brier_scores = brier_accuracy_scorer.score_many(df['two_year_recid'],df[predictor],df['race'])
    log_losses = log_loss_scorer.score_many(df['two_year_recid'],df[predictor],df['race'])
    calibration_gaps = get_sufficiency_gap(df['two_year_recid'],df[predictor],df['race'])
    sep_neg = get_separation_gap(df['two_year_recid'],df[predictor],df['race'],0)
    sep_pos = get_separation_gap(df['two_year_recid'],df[predictor],df['race'],1)
    for race in RACES:
        if race != 'all':
            table.add_row([predictor,race,brier_scores[race],log_losses[race],calibration_gaps[race],
                       sep_neg[race],sep_pos[race]])
        else:
            table.add_row([predictor,race,brier_scores[race],log_losses[race],float('nan'),float('nan'),float('nan')])
print(table.draw().replace('nan','N/A'))

Predictor   Race        Brier       Log Loss   Calibrati   Separatio   Separatio
                        Score                  on Gap      n Gap       n Gap    
                                                           (Negative   (Positive
                                                           Class)      Class)   
unconstra   all         0.210       0.607      N/A         N/A         N/A      
ined                                                                            
unconstra   African-    0.211       0.610      -0.021      0.040       0.034    
ined        American                                                            
unconstra   Caucasian   0.207       0.603      0.037       -0.042      -0.071   
ined                                                                            
calibrate   all         0.210       0.607      N/A         N/A         N/A      
d                                                                               
calibrate   African-    0.21

In [4]:
# for latex :~)
print(draw_latex(table).replace('nan','N/A'))

\begin{table}
	\begin{center}
		\begin{tabular}{lllllll}
			 \\
			\hline
			Predictor & Race & Brier Score & Log Loss & Calibration Gap & Separation Gap (Negative Class) & Separation Gap (Positive Class) \\
			unconstrained & all & 0.210 & 0.607 & N/A & N/A & N/A \\
			unconstrained & African-American & 0.211 & 0.610 & -0.021 & 0.040 & 0.034 \\
			unconstrained & Caucasian & 0.207 & 0.603 & 0.037 & -0.042 & -0.071 \\
			calibrated & all & 0.210 & 0.607 & N/A & N/A & N/A \\
			calibrated & African-American & 0.212 & 0.610 & -0.020 & 0.039 & 0.034 \\
			calibrated & Caucasian & 0.207 & 0.602 & 0.035 & -0.042 & -0.069 \\
			thresholdless_EO & all & 0.215 & 0.619 & N/A & N/A & N/A \\
			thresholdless_EO & African-American & 0.220 & 0.629 & -0.059 & 0.000 & -0.000 \\
			thresholdless_EO & Caucasian & 0.208 & 0.603 & 0.084 & -0.000 & 0.000 \\
		\end{tabular}
	\end{center}
\end{table}


In [14]:
# Heatmap of scores for different Brier weights
rows = []
for a,b in itertools.product(range(2,20),repeat=2):
    scores = {'a' : a , 'b' : b}
    for predictor in PREDICTOR_TYPES:
        scorer = WeightedBrierScorer(a,b)
        results = scorer.score_many(df['two_year_recid'],df[predictor],df['race'])
        for race in RACES:
            scores[predictor + '_' + race] = results[race]
    rows.append(scores)

scores_df = pd.DataFrame.from_dict(rows,orient='columns')
charts = []
for race in RACES:
    scores_df['cal_EO_pdiff_'+race] = scores_df[['calibrated_' + race,'thresholdless_EO_'+race]].pct_change(axis=1)['thresholdless_EO_'+race]
    chart = alt.Chart(scores_df).mark_rect().encode(
        x='b:O',
        y='a:O',
        color=alt.Color('cal_EO_pdiff_%s' % race,title='% Diff EO Calibrated')
    ).properties(title=race)
    charts.append(chart)
alt.concat(*charts).properties(title=alt.Title('Weighted Brier Scores',subtitle='Parameterized s₀=pᵃ, s₁=(1-p)ᵇ'))


alt.ConcatChart(...)

In [11]:
rows=[]
for a,b in itertools.product(range(1,20),repeat=2):
    scores = {'a' : a , 'b' : b}
    for predictor in PREDICTOR_TYPES:
        scorer = BetaScorer(a,b)
        results = scorer.score_many(df['two_year_recid'],df[predictor],df['race'])
        for race in RACES:
            scores[predictor + '_' + race] = results[race]
    rows.append(scores)

scores_df = pd.DataFrame.from_dict(rows,orient='columns')
charts = []
for race in RACES:
    scores_df['cal_EO_pdiff_'+race] = scores_df[['calibrated_' + race,'thresholdless_EO_'+race]].pct_change(axis=1)['thresholdless_EO_'+race]
    chart = alt.Chart(scores_df).mark_rect().encode(
        x='b:O',
        y='a:O',
        color=alt.Color('cal_EO_pdiff_%s' % race,title='% Diff EO Calibrated')
    ).properties(title=race)
    charts.append(chart)
alt.concat(*charts).properties(title=alt.Title('Beta scores',subtitle='Parameterized s₀=∫₀ᵖ tᵃ(1-t)ᵇ⁻¹ dt, s₁=∫ₚ¹tᵃ⁻¹(1-t)ᵇ dt'))


alt.ConcatChart(...)